I will now try to perform batch processing on input csv files, converting them to hdf5 using out-of-core processing.

In [1]:
%pylab
import DataSTORM.processors as proc
import DataSTORM.batch      as bat
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


Establish a filter and cleanup for testing.

In [2]:
filter1  = proc.Filter('loglikelihood', '<', 250)
cleanup  = proc.CleanUp()
pipeline = [cleanup, filter1]

In [3]:
inputDir = Path('../test-data/Centrioles/')
bp = bat.H5BatchProcessor(inputDir,
                          pipeline,
                          suffix = '_locResults_small.dat',
                          chunksize = None,
                          inputFileType = 'csv',
                          useSameFolder = True)

In [4]:
%time bp.go()

CPU times: user 604 ms, sys: 48 ms, total: 652 ms
Wall time: 651 ms


In [ ]:
%time bp.goMerge()

Frame 247: 43 trajectories present


So it looks like linking two datasets with 80,000 and 100,000 frames (and that are relatively sparse) will take approximately 12 hours when done in serial.

In [ ]:
test = Path('../test-data/Centrioles/FOV_1_noPB_1500mW_10ms_1/FOV_1_noPB_1500mW_10ms_1_MMStack_locResults_small_processed.h5')
test2 = pd.read_hdf(str(test), key = 'merged')

In [ ]:
test2.describe()

In [ ]:
import trackpy as tp

In [ ]:
filename = Path('../test-data/Centrioles/FOV_1_noPB_1500mW_10ms_1/FOV_1_noPB_1500mW_10ms_1_MMStack_locResults_small_processed.h5')
#store = tp.PandasHDFStoreSingleNode(str(filename), key = 'linked')
store = pd.read_hdf(str(filename), key = 'linked', iterator = True, chunksize = None)

In [ ]:
for ctr in store:
    print(ctr)

In [ ]:
test.agg({'loglikelihood' : 'mean', 'frame' : 'min'})

In [ ]:
temp = store.store.select_column('linked', 'particle').max()

In [ ]:
type(int(temp))